In [0]:
#!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [2]:
# Library loading
%matplotlib inline

import pandas as pd # manipulate dataframes
import matplotlib.pyplot as plt # plotting
import matplotlib
import numpy as np
import time

from sklearn.metrics import mean_squared_error
import h5py

# Check torch install
try:
  import torch
except:
  print("Starting a session, torch not installed, installing...")
  !pip3 install torch # we install torch if not installed
  import torch

# First we check if CUDA is available
print("CUDA AVAILABLE? ",torch.cuda.is_available())

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
      
device = get_default_device()
print(device)

CUDA AVAILABLE?  True
cuda


In [0]:
### Download datasets
downloaded = drive.CreateFile({'id':"1femDl-mpNeoLrbeBzfoLF3A_pNi2zR7D"})   # replace the id with id of file you want to access
downloaded.GetContentFile('NKAS_DataSet.hdf5')  

downloaded = drive.CreateFile({'id':"1s62a9Tfgmht0lUCjlAwVjW56vbngWE26"})   # replace the id with id of file you want to access
downloaded.GetContentFile('DataSet_0p20val.hdf5')  

downloaded = drive.CreateFile({'id':"1FxLvyBgmfQ17xctfOjzEvWcNmGa9F4sR"})   # replace the id with id of file you want to access
downloaded.GetContentFile('NKAS_density.hdf5')  

# Model function

In [0]:
class model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, nb_channels_raman,p_drop=0.5):
        super(model, self).__init__()
        
        # init parameters
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.num_layers  = num_layers
        self.nb_channels_raman = nb_channels_raman
        
        # network related torch stuffs
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=p_drop)

        self.linears = torch.nn.ModuleList([torch.nn.Linear(input_size, self.hidden_size)])
        self.linears.extend([torch.nn.Linear(self.hidden_size, self.hidden_size) for i in range(1, self.num_layers)])
            
#         self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)        
#         self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
#         self.fc3 = torch.nn.Linear(self.hidden_size, self.hidden_size)

        self.out_thermo = torch.nn.Linear(self.hidden_size, 6) # Linear output
        self.out_raman = torch.nn.Linear(self.hidden_size, self.nb_channels_raman) # Linear output
    
    def forward(self, x):
        """core neural network"""
        # Feedforward
        for layer in self.linears:
            x = self.dropout(self.relu(layer(x)))
        return x
#         hidden1 = self.fc1(x)
#         relu1 = self.dropout(self.relu(hidden1))
#         hidden2 = self.fc2(relu1)
#         relu2 = self.dropout(self.relu(hidden2))
#         hidden3 = self.fc3(relu2)
#         relu3 = self.dropout(self.relu(hidden3))        
#         return relu3 # return last layer
    
    def output_bias_init(self):
        """bias initialisation for self.out_thermo
        
        positions are Tg, Sconf(Tg), Ae, A_am, density, fragility (MYEGA one)
        """
        self.out_thermo.bias = torch.nn.Parameter(data=torch.tensor([np.log(1000.),np.log(10.),-1.5,-1.5,np.log(2.3),np.log(20.0)]))
    
    def at_gfu(self,x):
        """calculate atom per gram formula unit

        assumes rows are sio2 al2o3 na2o k2o
        """
        out = 3.0*x[:,0] + 5.0*x[:,1] + 3.0*x[:,2] + 3.0*x[:,3]
        return torch.reshape(out, (out.shape[0], 1))

    def aCpl(self,x):
        """calculate term a in equation Cpl = qCpl + bCpl*T
        """
        out = 81.37*x[:,0] + 27.21*x[:,1] + 100.6*x[:,2]  + 50.13*x[:,3] + x[:,0]*(x[:,3]*x[:,3])*151.7
        return torch.reshape(out, (out.shape[0], 1))

    def b_calc(self,x):
        """calculate term b in equation Cpl = aCpl + b*T
        """
        out = 0.09428*x[:,1] + 0.01578*x[:,3]
        return torch.reshape(out, (out.shape[0], 1))

    def ap_calc(self,x):
        """calculate term ap in equation dS = ap ln(T/Tg) + b(T-Tg)
        """
        out = self.aCpl(x) - 3.0*8.314462*self.at_gfu(x)
        return torch.reshape(out, (out.shape[0], 1))
    
    def dCp(self,x,T): 
        out = self.ap_calc(x)*(torch.log(T)-torch.log(self.tg(x))) + self.b_calc(x)*(T-self.tg(x))
        return torch.reshape(out, (out.shape[0], 1))
     
    def raman_pred(self,x):
        """Raman predicted spectra"""
        return self.out_raman(self.forward(x))
      
    def tg(self,x):
        """glass transition temperature Tg"""
        out = torch.exp(self.out_thermo(self.forward(x))[:,0])
        return torch.reshape(out, (out.shape[0], 1))
      
    def sctg(self,x):
        """configurational entropy at Tg"""
        out = torch.exp(self.out_thermo(self.forward(x))[:,1])
        return torch.reshape(out, (out.shape[0], 1))
    
    def ae(self,x):
        """Ae parameter in Adam and Gibbs and MYEGA"""
        out = self.out_thermo(self.forward(x))[:,2]
        return torch.reshape(out, (out.shape[0], 1))
      
    def a_am(self,x):
        """A parameter for Avramov-Mitchell"""
        out = self.out_thermo(self.forward(x))[:,3]
        return torch.reshape(out, (out.shape[0], 1))
      
    def density(self,x):
        """glass density"""
        out = torch.exp(self.out_thermo(self.forward(x))[:,4])
        return torch.reshape(out, (out.shape[0], 1))
    
    def fragility(self,x):
        """melt fragility"""
        out = torch.exp(self.out_thermo(self.forward(x))[:,5])
        return torch.reshape(out, (out.shape[0], 1))
    
    def be(self,x):
        """Be term in Adam-Gibbs eq given Ae, Tg and Scong(Tg)"""
        return (12.0-self.ae(x))*(self.tg(x)*self.sctg(x))
      
    def ag(self,x,T):
        """viscosity from the Adam-Gibbs equation, given chemistry X and temperature T
        """
        return self.ae(x) + self.be(x) / (T* (self.sctg(x) + self.dCp(x, T)))

    def myega(self,x, T):
        """viscosity frself.linears = nn.ModuleList([nn.Linear(input_size, layers_size)])
     self.linears.extend([nn.Linear(layers_size, layers_size) for i in range(1, self.num_layers-1)])
    om the MYEGA equation, given entries X and temperature T
        """
        return self.ae(x) + (12.0 - self.ae(x))*(self.tg(x)/T)*torch.exp((self.fragility(x)/(12.0-self.ae(x))-1.0)*(self.tg(x)/T-1.0))
      
    def am(self,x, T):
        """viscosity from the Avramov-Mitchell equation, given entries X and temperature T
        """
        return self.a_am(x) + (12.0 - self.a_am(x))*(self.tg(x)/T)**(self.fragility(x)/12.0)
    

# General function

In [0]:
def train_model(path_data,nb_neurons,nb_layers,p_drop, name, pretrain_patience = 500, train_patience=1000):

    path_raman = "./NKAS_DataSet.hdf5"
    path_density = "./NKAS_density.hdf5"

    f = h5py.File(path_data, 'r')
    
    # Entropy dataset
    X_entropy_train = f["X_entropy_train"].value
    y_entropy_train = f["y_entropy_train"].value

    X_entropy_valid = f["X_entropy_valid"].value
    y_entropy_valid = f["y_entropy_valid"].value

    X_entropy_test = f["X_entropy_test"].value
    y_entropy_test = f["y_entropy_test"].value

    # Viscosity dataset
    X_train = f["X_train"].value
    y_train = f["y_train"].value

    X_valid = f["X_valid"].value
    y_valid = f["y_valid"].value

    X_test = f["X_test"].value
    y_test = f["y_test"].value

    # Tg dataset
    X_tg_train = f["X_tg_train"].value
    X_tg_valid= f["X_tg_valid"].value
    X_tg_test = f["X_tg_test"].value

    y_tg_train = f["y_tg_train"].value
    y_tg_valid = f["y_tg_valid"].value
    y_tg_test = f["y_tg_test"].value

    f.close()

    # Raman dataset
    f = h5py.File(path_raman, 'r')
    X_raman_train = f["X_raman_train"].value
    y_raman_train = f["y_raman_train"].value
    X_raman_valid = f["X_raman_test"].value
    y_raman_valid = f["y_raman_test"].value
    f.close()

    # Density dataset
    f = h5py.File(path_density, 'r')
    X_density_train = f["X_density_train"].value
    X_density_valid = f["X_density_valid"].value
    X_density_test = f["X_density_test"].value

    y_density_train = f["y_density_train"].value
    y_density_valid = f["y_density_valid"].value
    y_density_test = f["y_density_test"].value
    f.close()

    # grabbing number of Raman channels
    nb_channels_raman = y_raman_valid.shape[1]

    # preparing data

    # viscosity
    x_visco_train = torch.FloatTensor(X_train[:,0:4]).to(device)
    T_visco_train = torch.FloatTensor(X_train[:,4].reshape(-1,1)).to(device)
    y_visco_train = torch.FloatTensor(y_train[:,0].reshape(-1,1)).to(device)

    x_visco_valid = torch.FloatTensor(X_valid[:,0:4]).to(device)
    T_visco_valid = torch.FloatTensor(X_valid[:,4].reshape(-1,1)).to(device)
    y_visco_valid = torch.FloatTensor(y_valid[:,0].reshape(-1,1)).to(device)

    # entropy
    x_entro_train = torch.FloatTensor(X_entropy_train[:,0:4]).to(device)
    y_entro_train = torch.FloatTensor(y_entropy_train[:,0].reshape(-1,1)).to(device)

    x_entro_valid = torch.FloatTensor(X_entropy_valid[:,0:4]).to(device)
    y_entro_valid = torch.FloatTensor(y_entropy_valid[:,0].reshape(-1,1)).to(device)

    # tg
    x_tg_train = torch.FloatTensor(X_tg_train[:,0:4]).to(device)
    y_tg_train = torch.FloatTensor(y_tg_train.reshape(-1,1)).to(device)

    x_tg_valid = torch.FloatTensor(X_tg_valid[:,0:4]).to(device)
    y_tg_valid = torch.FloatTensor(y_tg_valid.reshape(-1,1)).to(device)

    # Density
    x_density_train = torch.FloatTensor(X_density_train[:,0:4]).to(device)
    y_density_train = torch.FloatTensor(y_density_train.reshape(-1,1)).to(device)

    x_density_valid = torch.FloatTensor(X_density_valid[:,0:4]).to(device)
    y_density_valid = torch.FloatTensor(y_density_valid.reshape(-1,1)).to(device)

    # Raman
    x_raman_train = torch.FloatTensor(X_raman_train[:,0:4]).to(device)
    y_raman_train = torch.FloatTensor(y_raman_train).to(device)

    x_raman_valid = torch.FloatTensor(X_raman_valid[:,0:4]).to(device)
    y_raman_valid = torch.FloatTensor(y_raman_valid).to(device)

    # declaring model
    neuralmodel = model(4,nb_neurons,nb_layers,nb_channels_raman,p_drop=p_drop) 

    # criterion for match
    criterion = torch.nn.MSELoss()
    criterion.to(device)
    optimizer = torch.optim.Adam(neuralmodel.parameters(), lr = 0.001) # optimizer

    # we initialize the output bias
    neuralmodel.output_bias_init()

    # we send the neural net on device
    neuralmodel.to(device)
    
    #
    # PRETRAINING
    #
    neuralmodel.train()

    # for early stopping
    epoch = 0
    best_epoch = 0
    val_ex = 0

    # for recording losses
    record_pretrain_loss = []
    record_prevalid_loss = []

    while val_ex <= pretrain_patience:

        optimizer.zero_grad()

        # Forward pass
        y_raman_pred_train = neuralmodel.raman_pred(x_raman_train)
        y_density_pred_train = neuralmodel.density(x_density_train)
        y_tg_pred_train = neuralmodel.tg(x_tg_train)
        y_entro_pred_train = neuralmodel.sctg(x_entro_train)

        # on validation set
        y_raman_pred_valid = neuralmodel.raman_pred(x_raman_valid)
        y_density_pred_valid = neuralmodel.density(x_density_valid)
        y_tg_pred_valid = neuralmodel.tg(x_tg_valid)
        y_entro_pred_valid = neuralmodel.sctg(x_entro_valid)

        # Compute Loss

        # train 
        loss_tg = criterion(y_tg_pred_train, y_tg_train)
        loss_raman = criterion(y_raman_pred_train,y_raman_train)
        loss_density = criterion(y_density_pred_train,y_density_train)
        loss_entro = criterion(y_entro_pred_train,y_entro_train)

        loss = 0.001*loss_tg + loss_entro + 10*loss_raman + 1000*loss_density 

        # validation
        loss_tg_v = criterion(y_tg_pred_valid, y_tg_valid)
        loss_raman_v = criterion(y_raman_pred_valid,y_raman_valid)
        loss_density_v = criterion(y_density_pred_valid,y_density_valid)
        loss_entro_v = criterion(y_entro_pred_valid,y_entro_valid)

        loss_v = 0.001*loss_tg_v + loss_entro_v + 10*loss_raman_v + 1000*loss_density_v 

        record_pretrain_loss.append(loss.item())
        record_prevalid_loss.append(loss_v.item())

        # calculating early-stopping criterion
        if epoch == 0:
            val_ex = 0
            best_loss_v = loss_v.item()
        elif loss_v.item() <= best_loss_v:
            val_ex = 0
            best_epoch = epoch
            best_loss_v = loss_v.item()
        else:
            val_ex += 1

        # Backward pass
        loss.backward()
        optimizer.step()

        epoch += 1
                

    #
    # TRAINING
    #
    neuralmodel.train()

    # for early stopping
    epoch = 0
    best_epoch = 0
    val_ex = 0

    # for recording losses
    record_train_loss = []
    record_valid_loss = []
    
    while val_ex <= train_patience:
      
        optimizer.zero_grad()

        # Forward pass
        y_ag_pred_train = neuralmodel.ag(x_visco_train,T_visco_train)
        y_myega_pred_train = neuralmodel.myega(x_visco_train,T_visco_train)
        y_am_pred_train = neuralmodel.am(x_visco_train,T_visco_train)
        y_raman_pred_train = neuralmodel.raman_pred(x_raman_train)
        y_density_pred_train = neuralmodel.density(x_density_train)
        y_entro_pred_train = neuralmodel.sctg(x_entro_train)

        # on validation set
        y_ag_pred_valid = neuralmodel.ag(x_visco_valid,T_visco_valid)
        y_myega_pred_valid = neuralmodel.myega(x_visco_valid,T_visco_valid)
        y_am_pred_valid = neuralmodel.am(x_visco_valid,T_visco_valid)
        y_raman_pred_valid = neuralmodel.raman_pred(x_raman_valid)
        y_density_pred_valid = neuralmodel.density(x_density_valid)
        y_entro_pred_valid = neuralmodel.sctg(x_entro_valid)

        # Compute Loss

        # train 
        loss_ag = criterion(y_ag_pred_train, y_visco_train)
        loss_myega = criterion(y_myega_pred_train, y_visco_train)
        loss_am = criterion(y_am_pred_train, y_visco_train)
        loss_raman = criterion(y_raman_pred_train,y_raman_train)
        loss_density = criterion(y_density_pred_train,y_density_train)
        loss_entro = criterion(y_entro_pred_train,y_entro_train)

        loss = loss_entro + loss_ag + loss_myega + loss_am + 10*loss_raman + 1000*loss_density 

        # validation
        loss_ag_v = criterion(y_ag_pred_valid, y_visco_valid)
        loss_myega_v = criterion(y_myega_pred_valid, y_visco_valid)
        loss_am_v = criterion(y_am_pred_valid, y_visco_valid)
        loss_raman_v = criterion(y_raman_pred_valid,y_raman_valid)
        loss_density_v = criterion(y_density_pred_valid,y_density_valid)
        loss_entro_v = criterion(y_entro_pred_valid,y_entro_valid)

        loss_v = loss_entro_v + loss_ag_v + loss_myega_v + loss_am_v + 10*loss_raman_v + 1000*loss_density_v

        record_train_loss.append(loss.item())
        record_valid_loss.append(loss_v.item())

        # calculating ES criterion
        if epoch == 0:
            val_ex = 0
            best_loss_v = loss_v.item()
            # save best model
            torch.save(neuralmodel.state_dict(), name)
        elif loss_v.item() <= best_loss_v:
            val_ex = 0
            best_epoch = epoch
            best_loss_v = loss_v.item()

            # save best model
            torch.save(neuralmodel.state_dict(), name)
        else:
            val_ex += 1

        # Backward pass
        loss.backward()
        optimizer.step()

        epoch += 1
        
    #
    # SAVING MODEL
    #
    
    file_out = drive.CreateFile()
    # Read file and set it as a content of this instance.
    file_out.SetContentFile(name)
    file_out.Upload() # Upload the file.
    print('Model {} with valid loss {} saved'.format(name,loss_v.item()))

In [15]:
path_data = "./DataSet_0p20val.hdf5"

nb_exp = 10
nb_neurons = 300
nb_layers = 3
p_drop = 0.3

for i in range(nb_exp):
    print('Experiment {} started...'.format(i))
    
    path_out = "loss_ag_myega_am_ram_d_"+str(i)
    
    train_model(path_data,nb_neurons,nb_layers,p_drop, path_out, pretrain_patience = 200, train_patience=200)

Experiment 0 started...
Model loss_ag_myega_am_ram_d_0 with valid loss 4.101059913635254 saved
Experiment 1 started...
Model loss_ag_myega_am_ram_d_1 with valid loss 4.481479644775391 saved
Experiment 2 started...
Model loss_ag_myega_am_ram_d_2 with valid loss 4.885275363922119 saved
Experiment 3 started...
Model loss_ag_myega_am_ram_d_3 with valid loss 4.606471061706543 saved
Experiment 4 started...
Model loss_ag_myega_am_ram_d_4 with valid loss 4.100840091705322 saved
Experiment 5 started...
Model loss_ag_myega_am_ram_d_5 with valid loss 4.145567417144775 saved
Experiment 6 started...
Model loss_ag_myega_am_ram_d_6 with valid loss 19.383668899536133 saved
Experiment 7 started...
Model loss_ag_myega_am_ram_d_7 with valid loss 4.690925121307373 saved
Experiment 8 started...
Model loss_ag_myega_am_ram_d_8 with valid loss 5.580639839172363 saved
Experiment 9 started...
Model loss_ag_myega_am_ram_d_9 with valid loss 4.376068592071533 saved


In [0]:
p_drop


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])